<a href="https://colab.research.google.com/github/tolgakurtuluss/llmeval_test/blob/main/tolega_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lighteval

In [14]:
import json
import pandas as pd
import uuid
import random

# JSON dosyasını oku
with open('/content/data.txt', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Örnekleri al
examples = data['examples']

# DataFrame için veri hazırla
df_data = []

for i, example in enumerate(examples):
    question = example['input'].split('\n')[0]
    options = list(example['target_scores'].keys())

    row = {
        'context': question,
        'choice_a': options[0] if len(options) > 0 else '',
        'choice_b': options[1] if len(options) > 1 else '',
        'choice_c': options[2] if len(options) > 2 else '',
        'choice_d': options[3] if len(options) > 3 else '',
        'choice_e': options[4] if len(options) > 4 else '',
        'target': list(example['target_scores'].values())
    }

    df_data.append(row)

# DataFrame oluştur
df = pd.DataFrame(df_data)

# String sütunlar için uzunluk bilgisi ekle
string_columns = ['context', 'choice_a', 'choice_b', 'choice_c', 'choice_d', 'choice_e']

# Target sütununu düzenle
def map_target(x):
    index = x.index(1) if 1 in x else -1
    return {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E'}.get(index, 'Invalid')

df['target'] = df['target'].apply(map_target)

# Sonucu göster
print(df)

# İsteğe bağlı: CSV olarak kaydet
df.to_csv('bilgi_zorunlulugu_degerlendirme.csv', index=False)

                                              context  \
0   Ayşe fil mi yoksa zebra mı daha çok sever? Aşa...   
1   Bir sınıftaki tüm öğrenciler bir sınava girdi....   
2   Samet hangi yöne bakıyor? Aşağıdaki ifadelerde...   
3   Sepette sadece mavi toplar var mı? Aşağıdaki i...   
4   Ayşe havuç sever mi? Aşağıdaki ifadelerden han...   
5   B olayının gerçekleşme olasılığı nedir? Aşağıd...   
6   İnşaat müteahhidi özel bir lisansa ihtiyaç duy...   
7   Bir duvarı 10 işçi 6 saatte örer. Duvarı 5 tuğ...   
8   Bir öğrenci matematik ve fen derslerinden sına...   
9   Bir kitaplıkta her rafta eşit sayıda kitap var...   
10  Bir kutuda farklı renklerde kalemler var. Kutu...   
11  Bir araba saatte 60 km hızla giderse, 3 saatte...   
12  Bir çiftlikteki tavuk ve ineklerin toplam ayak...   

                                            choice_a  \
0   Sadece 1. ifade yeterlidir, 2. ifade yetersizdir   
1   Sadece 1. ifade yeterlidir, 2. ifade yetersizdir   
2   Sadece 1. ifade yeterlidir, 2

In [53]:
# Gerekli kütüphaneleri içe aktarın
import pandas as pd
from datasets import Dataset
from huggingface_hub import HfApi, notebook_login

# 1. Adım: Hugging Face'e giriş yapın
notebook_login()

# 2. Adım: DataFrame'i Dataset'e dönüştürün
dataset = Dataset.from_pandas(df)

# 3. Adım: Dataset'i push edin
dataset.push_to_hub("tolgadev/eval_tk")

# 4. Adım: README dosyası oluşturun (isteğe bağlı)
readme_content = """
# Dataset Adı

Bu dataset, bilgi zorunluluğu değerlendirme görevleri için oluşturulmuştur.

## İçerik

Dataset şu sütunları içermektedir:
- context: Soru metni
- choice_a, choice_b, choice_c, choice_d, choice_e: Cevap seçenekleri
- target: Doğru cevabın harfi (A, B, C, D, E)
- context_uuid: Benzersiz tanımlayıcı
- wiki_id: Rastgele atanmış ID

## Kullanım

Bu dataset, bilgi zorunluluğu değerlendirme modelleri eğitmek ve test etmek için kullanılabilir.

## Oluşturma

Bu dataset, [kaynak_adi] verisinden oluşturulmuştur.

## Lisans

[Uygun lisans bilgisini ekleyin]
"""

# README.md dosyasını oluşturun
with open("README.md", "w") as f:
    f.write(readme_content)

# README dosyasını Hugging Face'e yükleyin
api = HfApi()
api.upload_file(
    path_or_fileobj="README.md",
    path_in_repo="README.md",
    repo_id="tolgadev/eval_tk",
    repo_type="dataset"
)

print("Dataset başarıyla Hugging Face'e yüklendi!")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/hf_api.py:3757: UserWarning: Warnings while validating metadata in README.md:
- empty or missing yaml metadata in repo card
  warnings.warn(f"Warnings while validating metadata in README.md:\n{message}")


Dataset başarıyla Hugging Face'e yüklendi!


In [24]:
# MIT License

# Copyright (c) 2024 The HuggingFace Team
# Copyright (c) 2024 Philip May, Deutsche Telekom AG

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
# SOFTWARE.

# ruff: noqa: F405, F403, F401
"""
Custom evaluation tasks for lighteval.

This file generally creates just a TASKS_TABLE and TASKS_GROUPS which are then imported by LightEval.
This module implements the 4 tasks of deutsche-telekom/Ger-RAG-eval.
See: https://huggingface.co/datasets/deutsche-telekom/Ger-RAG-eval
"""

from lighteval.metrics.metrics import Metrics
from lighteval.tasks.lighteval_task import LightevalTaskConfig
from lighteval.tasks.requests import Doc


def prompt_fn_choose_question_by_context(line, task_name: str = None):
    instruction = "Aşağıdaki sorulardan hangisi (A veya B veya C veya D veya E) verilen bağlama göre cevaplanabilir?\n\n"
    query_template = """\
İçerik:
{context}

Seçenekler:
A: {choice_a}
B: {choice_b}
C: {choice_c}
D: {choice_d}
E: {choice_e}

Cevap:"""
    query = instruction + query_template.format(
        context=line["context"],
        choice_a=line["choice_a"],
        choice_b=line["choice_b"],
        choice_c=line["choice_c"],
        choice_d=line["choice_d"],
        choice_e=line["choice_e"],
    )
    choices = ["A", "B", "C", "D", "E"]
    return Doc(
        task_name=task_name,
        instruction=instruction,
        query=query,
        choices=choices,
        gold_index=choices.index(line["target"]),
    )

# Task 1: Choose question by context.
# Given is a context and 5 questions.
# The task is to decide which question can be answered by the context.
task1 = LightevalTaskConfig(
    name="tolega_eval:choose_question_by_context",
    prompt_function=prompt_fn_choose_question_by_context,
    suite=["community"],
    hf_repo="tolgadev/eval_tk",
    hf_subset="task1",
    hf_avail_splits=["train"],
    evaluation_splits=["train"],
    few_shots_split="train",
    few_shots_select="sequential",
    metric=[Metrics.loglikelihood_acc],
    version=1,
)

# STORE YOUR EVALS
TASKS_TABLE = [task1]


# MODULE LOGIC
# You should not need to touch this

if __name__ == "__main__":
    print(t.name for t in TASKS_TABLE)
    print(len(TASKS_TABLE))

<generator object <genexpr> at 0x7d4815d79690>
1


In [43]:
from datasets import load_dataset
cache_dir = '.'
dataset = load_dataset('tolgadev/eval_tk', split='train', cache_dir=cache_dir)

Repo card metadata block was not found. Setting CardData to empty.


train-00000-of-00001.parquet:   0%|          | 0.00/6.40k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13 [00:00<?, ? examples/s]

In [56]:
!export HF_DATASETS_TRUST_REMOTE_CODE=TRUE

# If 'task1' is not a valid dataset config, use 'default' or the correct config
!lighteval accelerate \
    --model_args "pretrained=google/gemma-2-2b,model_parallel=True,trust_remote_code=True" \
    --use_chat_template \
    --override_batch_size 1 \
    --tasks "/content/tasks/tolega_task.txt" \
    --custom_tasks "/content/community_tasks /tolega_eval.py" \
    --output_dir="./evals/"

2024-09-29 14:50:09.878406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 14:50:09.917979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 14:50:09.929551: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 14:50:12.238138: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
INFO:absl:Using default tokenizer.
Loading checkpoint shards: 100% 3/3 [00:01<00:00,  2.72it/s]
Repo 